In [5]:
import pandas as pd

df = pd.read_json("hf://datasets/HumanLLMs/Human-Like-DPO-Dataset/data.json")

1. Complex Word Ratio 비교

2. Measure of Textual Lexical Diversity

텍스트가 특정 TTR 기준에 도달할 때 까지 평균적으로 필요한 단어 계산

In [6]:
pip install lexical-diversity

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: C:\Users\strow\AppData\Local\Programs\Python\Python313\python.exe -m pip install --upgrade pip


In [7]:
from lexical_diversity import lex_div as ld
from scipy import stats
import numpy as np
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\strow\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\strow\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [8]:


# NLTK 데이터 다운로드
nltk.data.find('tokenizers/punkt')

def preprocess_for_lexdiv(text):
    #정제된 단어 리스트 만들기
    tokens = nltk.word_tokenize(str(text).lower())
    words = [word for word in tokens if word.isalpha()] # 구두점, 숫자 제거
    return words

MTLD_scores = pd.DataFrame()

print("MTLD 계산을 위해 텍스트 전처리 중...")
# 1. 'chosen', 'rejected' 텍스트를 [단어, 리스트]로 변환
MTLD_scores['chosen_words'] = df['chosen'].apply(preprocess_for_lexdiv)
MTLD_scores['rejected_words'] = df['rejected'].apply(preprocess_for_lexdiv)

print("MTLD 점수 계산 중... (행이 많으면 시간이 걸릴 수 있습니다)")
# 2. MTLD 계산
MTLD_scores['chosen_mtld'] = MTLD_scores['chosen_words'].apply(ld.mtld)
MTLD_scores['rejected_mtld'] = MTLD_scores['rejected_words'].apply(ld.mtld)
MTLD_scores['mtld_diff'] = MTLD_scores['chosen_mtld'] - MTLD_scores['rejected_mtld']

print("MTLD 점수 계산 완료. 결과 요약:")
print(MTLD_scores[['chosen_mtld', 'rejected_mtld', 'mtld_diff']].describe())

MTLD 계산을 위해 텍스트 전처리 중...
MTLD 점수 계산 중... (행이 많으면 시간이 걸릴 수 있습니다)
MTLD 점수 계산 완료. 결과 요약:
        chosen_mtld  rejected_mtld     mtld_diff
count  10884.000000   10884.000000  10884.000000
mean      91.677915      86.910725      4.767190
std       21.080165      23.955348     29.025899
min        0.000000       0.000000   -173.551304
25%       77.674862      70.378573    -13.287804
50%       90.171135      84.241585      5.540390
75%      104.104811     100.304613     23.784791
max      383.320000     272.160000    306.320000


In [9]:
MTLD_scores = MTLD_scores.drop(columns=['chosen_words', 'rejected_words'])
MTLD_scores.to_csv('mtld_results.csv', index=False, encoding='utf-8-sig')

In [10]:
from scipy import stats

# NaN 값 제거 
paired_scores = MTLD_scores[['chosen_mtld', 'rejected_mtld']].dropna()

# Paired T-Test
statistic, pvalue = stats.ttest_rel(paired_scores['chosen_mtld'], 
                                  paired_scores['rejected_mtld'])

print("\n--- MTLD (rejected vs chosen) ---")
print(f"  T-statistic: {statistic:.4f}")
print(f"  P-value: {pvalue:.6f}") 

# 결과 해석
alpha = 0.05
if pvalue < alpha:
    print(f"\n결론: P-value({pvalue:.6f}) < {alpha} 이므로,")
    print("  'chosen'과 'rejected'의 어휘 다양성(MTLD) 차이는 통계적으로 유의미합니다.")
else:
    print(f"\n결론: P-value({pvalue:.6f}) >= {alpha} 이므로,")
    print("  'chosen'과 'rejected'의 어휘 다양성(MTLD) 차이는 유의미하지 않습니다.")

# 4. .describe() 결과와 비교
mean_diff = MTLD_scores['mtld_diff'].mean()
print(f"\n참고: 평균 MTLD 차이 (Chosen - Rejected): {mean_diff:.4f}")

if mean_diff > 0:
    print("  (평균적으로 'Chosen' 텍스트의 MTLD 점수가 더 높습니다.)")
elif mean_diff < 0:
    print("  (평균적으로 'Rejected' 텍스트의 MTLD 점수가 더 높습니다.)")


--- MTLD (rejected vs chosen) ---
  T-statistic: 17.1345
  P-value: 0.000000

결론: P-value(0.000000) < 0.05 이므로,
  'chosen'과 'rejected'의 어휘 다양성(MTLD) 차이는 통계적으로 유의미합니다.

참고: 평균 MTLD 차이 (Chosen - Rejected): 4.7672
  (평균적으로 'Chosen' 텍스트의 MTLD 점수가 더 높습니다.)
